# Strands Agents 응답의 고급 처리

Strands Agents를 사용하면 에이전트 실행 중에 발생하는 이벤트를 두 가지 방법으로 가로채고 처리할 수 있습니다:
    
- **비동기 반복자(Async iterators)**: FastAPI, aiohttp 또는 Django Channels와 같은 비동기 프레임워크에 이상적입니다. 이러한 환경을 위해 SDK는 비동기 반복자를 반환하는 `stream_async` 메서드를 제공합니다.
- **콜백 핸들러(Callback handlers)**: 에이전트 실행 중에 발생하는 이벤트를 가로채고 처리할 수 있게 해줍니다. 이를 통해 실시간 모니터링, 사용자 정의 출력 형식 지정 및 외부 시스템과의 통합이 가능합니다.

이 예제에서는 에이전트 호출을 처리하기 위해 두 방법을 모두 사용하는 방법을 보여드리겠습니다.


## 에이전트 세부사항
<div style="float: left; margin-right: 20px; ">
    
|기능                |설명                                               |
|--------------------|---------------------------------------------------|
|사용된 기능          |비동기 반복자, 콜백 핸들러                          |
|에이전트 구조        |단일 에이전트 아키텍처                              |
|사용된 네이티브 도구 |calculator                                         |
|생성된 사용자 정의 도구|날씨 예보                                          |

</div>

## 아키텍처

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

## 주요 기능
* 스트리밍을 위한 비동기 반복자
* 콜백 핸들러


## 설정 및 사전 요구사항

### 사전 요구사항
* Python 3.10+
* AWS 계정
* Amazon Bedrock에서 Anthropic Claude 3.7 활성화

이제 Strands Agent에 필요한 패키지들을 설치하겠습니다

In [ ]:
# 사전 요구사항 설치
!pip install -r requirements.txt

### 의존성 패키지 가져오기

이제 의존성 패키지들을 가져오겠습니다

In [ ]:
import asyncio

import httpx
import nest_asyncio
import uvicorn
from fastapi import FastAPI
from fastapi.responses import StreamingResponse
from pydantic import BaseModel
from strands import Agent, tool
from strands_tools import calculator

## 방법 1 - 스트리밍을 위한 비동기 반복자


Strands Agents는 `stream_async` 메서드를 통해 비동기 반복자를 지원하여 웹 서버, API 및 기타 비동기 애플리케이션과 같은 비동기 환경에서 에이전트 응답의 실시간 스트리밍을 가능하게 합니다.

노트북에서 이 예제를 보여주고 있으므로 `asyncio.run`과 `loop.run_until_complete`의 중첩 사용을 허용하기 위해 `nest_asyncio`를 적용해야 합니다.

In [ ]:
nest_asyncio.apply()

### stream_async로 에이전트 생성 및 호출

이제 내장 계산기 도구와 `callback_handler` 없이 에이전트를 생성하겠습니다. `stream_async` 메서드를 사용하여 스트리밍된 에이전트 이벤트를 반복하겠습니다.

In [ ]:
# 콜백 핸들러 없이 에이전트 초기화
agent = Agent(
    model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
    tools=[calculator], 
    callback_handler=None)

# 스트리밍된 에이전트 이벤트를 반복하는 비동기 함수


async def process_streaming_response():
    agent_stream = agent.stream_async("2+2를 계산해주세요")
    async for event in agent_stream:
        print(event)


# 에이전트 실행
asyncio.run(process_streaming_response())

###  이벤트 루프 생명주기 추적

이 예제는 이벤트 루프 생명주기와 이벤트들이 서로 어떻게 관련되는지를 보여줍니다. Strands Agent의 실행 흐름을 이해하는 데 유용합니다:

에이전트 스트림 이벤트를 더 잘 분석하기 위해 출력 형식 코드를 만들어보겠습니다. 동일한 에이전트를 계속 사용하겠습니다.

In [ ]:
# 스트리밍된 에이전트 이벤트를 반복하는 비동기 함수


async def process_streaming_response():
    agent_stream = agent.stream_async("프랑스의 수도는 어디이고 42+7은 얼마인가요?")
    async for event in agent_stream:
        # 이벤트 루프 생명주기 추적
        if event.get("init_event_loop", False):
            print("🔄 이벤트 루프 초기화됨")
        elif event.get("start_event_loop", False):
            print("▶️ 이벤트 루프 사이클 시작")
        elif event.get("start", False):
            print("📝 새 사이클 시작됨")
        elif "message" in event:
            print(f"📬 새 메시지 생성됨: {event['message']['role']}")
        elif event.get("force_stop", False):
            print(
                f"🛑 이벤트 루프 강제 중지됨: {event.get('force_stop_reason', '알 수 없는 이유')}"
            )

        # 도구 사용 추적
        if "current_tool_use" in event and event["current_tool_use"].get("name"):
            tool_name = event["current_tool_use"]["name"]
            print(f"🔧 도구 사용 중: {tool_name}")

        # 출력을 깔끔하게 유지하기 위해 텍스트의 일부만 표시
        if "data" in event:
            # 데모 목적으로 각 청크의 처음 20자만 표시
            data_snippet = event["data"][:20] + (
                "..." if len(event["data"]) > 20 else ""
            )
            print(f"📟 텍스트: {data_snippet}")

    return event["result"]


# 에이전트 실행
asyncio.run(process_streaming_response())

### FastAPI 통합

`stream_async`를 FastAPI와 통합하여 애플리케이션에 스트리밍 엔드포인트를 만들 수도 있습니다. 이를 위해 에이전트에 `weather_forecast` 도구를 추가하겠습니다. 아키텍처 업데이트는 다음과 같습니다:

<div style="text-align:left;">
    <img src="images/architecture_2.png" width="65%" />
</div>

In [ ]:
# 도구 정의


@tool
def weather_forecast(city: str, days: int = 3) -> str:
    """지정된 도시의 날씨 예보를 가져옵니다.
    
    Args:
        city: 도시 이름
        days: 예보 일수 (기본값: 3일)
    
    Returns:
        날씨 예보 정보
    """
    return f"{city}의 향후 {days}일간 날씨 예보..."


# FastAPI 앱
app = FastAPI()


class PromptRequest(BaseModel):
    prompt: str


@app.post("/stream")
async def stream_response(request: PromptRequest):
    async def generate():
        agent = Agent(tools=[calculator, weather_forecast], callback_handler=None)
        try:
            async for event in agent.stream_async(request.prompt):
                if "data" in event:
                    yield event["data"]
        except Exception as e:
            yield f"오류: {str(e)}"

    return StreamingResponse(generate(), media_type="text/plain")


# 블로킹 없이 서버를 시작하는 함수


async def start_server():
    config = uvicorn.Config(app, host="0.0.0.0", port=8001, log_level="info")
    server = uvicorn.Server(config)
    await server.serve()


# 서버를 백그라운드 작업으로 실행
if "server_task" not in globals():
    server_task = asyncio.create_task(start_server())
    await asyncio.sleep(0.1)  # 서버가 시작할 시간을 줌

print("✅ 서버가 http://0.0.0.0:8001에서 실행 중입니다")

#### FastAPI 에이전트 호출
이제 프롬프트로 에이전트를 호출할 수 있습니다

In [ ]:
async def fetch_stream():
    async with httpx.AsyncClient() as client:
        async with client.stream(
            "POST",
            "http://0.0.0.0:8001/stream",
            json={"prompt": "뉴욕의 날씨는 어떤가요?"},
        ) as response:
            async for line in response.aiter_lines():
                if line.strip():  # 빈 줄 건너뛰기
                    print("수신됨:", line)


await fetch_stream()

## 방법 2 - 스트리밍을 위한 콜백 핸들러

콜백 핸들러는 Strands Agents의 강력한 기능으로, 에이전트 실행 중에 발생하는 이벤트를 가로채고 처리할 수 있게 해줍니다. 이를 통해 실시간 모니터링, 사용자 정의 출력 형식 지정 및 외부 시스템과의 통합이 가능합니다.



콜백 핸들러는 에이전트의 생명주기 동안 발생하는 이벤트를 실시간으로 수신합니다:

- 모델의 텍스트 생성
- 도구 선택 및 실행
- 추론 과정
- 오류 및 완료


이제 도구 사용과 모델 출력을 강조하기 위해 이벤트 입력을 형식화하는 사용자 정의 콜백 핸들러 함수를 만들어보겠습니다. 이를 위해 계산기 도구만 있는 에이전트를 다시 사용하겠습니다.

<div style="text-align:left;">
    <img src="images/architecture.png" width="65%" />
</div>

In [ ]:
def custom_callback_handler(**kwargs):
    """사용자 정의 콜백 핸들러 함수"""
    # 스트림 데이터 처리
    if "data" in kwargs:
        print(f"모델 출력: {kwargs['data']}")
    elif "current_tool_use" in kwargs and kwargs["current_tool_use"].get("name"):
        print(f"\n도구 사용 중: {kwargs['current_tool_use']['name']}")


# 사용자 정의 콜백 핸들러로 에이전트 생성
agent = Agent(   
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",  # 선택사항: 모델 ID 지정
        tools=[calculator], 
        callback_handler=custom_callback_handler)

agent("2+2를 계산해주세요")

### 축하합니다!

이 노트북에서 비동기 반복자와 콜백 핸들러를 사용하여 에이전트 출력을 스트리밍하는 방법을 배웠습니다.